# Robot Data Inspection

This notebook inspects the processed robot data from timer task recordings.


In [1]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
import cv2
from pathlib import Path
import json

# Set up matplotlib for better display
plt.rcParams['figure.figsize'] = (15, 10)
plt.rcParams['font.size'] = 12


In [2]:
# Robot data file path
hdf5_path = "YourPATH.hdf5"
print(f"Checking robot data file: {hdf5_path}")
print(f"File exists: {Path(hdf5_path).exists()}")

if Path(hdf5_path).exists():
    # Open HDF5 file
    with h5py.File(hdf5_path, 'r') as f:
        print("\n=== HDF5 File Structure ===")
        
        def print_structure(name, obj):
            print(name)
        
        f.visititems(print_structure)
        
        print("\n=== File Attributes ===")
        for key in f.attrs:
            print(f"{key}: {f.attrs[key]}")
else:
    print("Robot data file not found! Please check the path.")


Checking robot data file: YourPATH.hdf5
File exists: False
Robot data file not found! Please check the path.


In [3]:
if Path(hdf5_path).exists():
    with h5py.File(hdf5_path, 'r') as f:
        print("=== Robot Data Content Analysis ===")
        
        # Check if it's in robomimic format (with data group)
        if 'data' in f:
            print("\n--- RoboMimic Format Detected ---")
            data_group = f['data']
            demo_keys = list(data_group.keys())
            print(f"Number of robot demos: {len(demo_keys)}")
            
            if demo_keys:
                # Inspect first demo
                first_demo = demo_keys[0]
                demo = data_group[first_demo]
                print(f"\n--- First Robot Demo: {first_demo} ---")
                
                # Print demo structure
                def print_demo_structure(name, obj):
                    if isinstance(obj, h5py.Dataset):
                        print(f"  {name}: shape={obj.shape}, dtype={obj.dtype}")
                    else:
                        print(f"  {name}: group")
                
                demo.visititems(print_demo_structure)
                
                # Print demo attributes
                print(f"\nDemo attributes:")
                for key in demo.attrs:
                    print(f"  {key}: {demo.attrs[key]}")
        
        # Check if it's raw RealSense format
        elif 'rgb' in f:
            print("\n--- Raw RealSense Format Detected ---")
            print(f"RGB shape: {f['rgb'].shape}")
            print(f"Depth shape: {f['depth'].shape}")
            print(f"Timestamps shape: {f['timestamps'].shape}")
            
            # Print attributes
            print("\nFile attributes:")
            for key in f.attrs:
                print(f"  {key}: {f.attrs[key]}")
        
        else:
            print("\n--- Unknown Format ---")
            print("Available top-level keys:", list(f.keys()))


In [4]:
if Path(hdf5_path).exists():
    with h5py.File(hdf5_path, 'r') as f:
        if 'data' in f:
            data_group = f['data']
            demo_keys = list(data_group.keys())
            
            if demo_keys:
                # Get first demo
                first_demo = demo_keys[0]
                demo = data_group[first_demo]
                
                # Check available camera types
                camera_types = []
                if 'obs/front_img_1' in demo:
                    camera_types.append('front_img_1')
                if 'obs/left_wrist_img' in demo:
                    camera_types.append('left_wrist_img')
                if 'obs/right_wrist_img' in demo:
                    camera_types.append('right_wrist_img')
                # Check available image processing types for front camera
                front_img_types = []
                if 'obs/front_img_1' in demo:
                    front_img_types.append('front_img_1')
                if 'obs/front_img_1_masked' in demo:
                    front_img_types.append('front_img_1_masked')
                if 'obs/front_img_1_line' in demo:
                    front_img_types.append('front_img_1_line')
                
                print(f"Available robot camera types: {camera_types}")
                print(f"Available front camera image types: {front_img_types}")
                
                # Frame selection: start from frame 0, every 50 steps, show 5 frames
                frame_step = 50
                num_frames_to_show = 5
                total_frames = demo['obs/front_img_1'].shape[0] if 'obs/front_img_1' in demo else 0
                
                # Calculate frame indices
                frame_indices = [i * frame_step for i in range(num_frames_to_show) if i * frame_step < total_frames]
                actual_frames = len(frame_indices)
                
                print(f"Total frames available: {total_frames}")
                print(f"Showing frames: {frame_indices}")
                
                if actual_frames > 0:
                    # Display basic camera images (3 rows: front, left, right)
                    print("\n=== Basic Camera Images ===")
                    fig1, axes1 = plt.subplots(len(camera_types), actual_frames, figsize=(20, 12))
                    if len(camera_types) == 1:
                        axes1 = axes1.reshape(1, -1)
                    if actual_frames == 1:
                        axes1 = axes1.reshape(-1, 1)
                    
                    for cam_idx, camera_type in enumerate(camera_types):
                        print(f"\n=== {camera_type.upper()} Camera ===")
                        
                        images = demo[f'obs/{camera_type}']
                        print(f"Shape: {images.shape}")
                        
                        for frame_idx, frame_num in enumerate(frame_indices):
                            img_display = images[frame_num]
                            axes1[cam_idx, frame_idx].imshow(img_display)
                            axes1[cam_idx, frame_idx].set_title(f'{camera_type}\nFrame {frame_num}')
                            axes1[cam_idx, frame_idx].axis('off')
                    
                    plt.tight_layout()
                    plt.show()
                    
                    # Display front camera processed images (masked and line)
                    if len(front_img_types) > 1:  # More than just basic front_img_1
                        print("\n=== Front Camera Processed Images ===")
                        fig2, axes2 = plt.subplots(len(front_img_types), actual_frames, figsize=(20, 8))
                        if len(front_img_types) == 1:
                            axes2 = axes2.reshape(1, -1)
                        if actual_frames == 1:
                            axes2 = axes2.reshape(-1, 1)
                        
                        for img_idx, img_type in enumerate(front_img_types):
                            print(f"\n=== {img_type.upper()} ===")
                            
                            images = demo[f'obs/{img_type}']
                            print(f"Shape: {images.shape}")
                            
                            for frame_idx, frame_num in enumerate(frame_indices):
                                img_display = images[frame_num]
                                axes2[img_idx, frame_idx].imshow(img_display)
                                axes2[img_idx, frame_idx].set_title(f'{img_type}\nFrame {frame_num}')
                                axes2[img_idx, frame_idx].axis('off')
                        
                        plt.tight_layout()
                        plt.show()
                else:
                    print("No frames available for display")
        
        else:
            print("No robomimic format robot data found")


In [5]:
if Path(hdf5_path).exists():
    with h5py.File(hdf5_path, 'r') as f:
        if 'data' in f:
            data_group = f['data']
            demo_keys = list(data_group.keys())
            
            if demo_keys:
                print("=== Robot Actions and Poses Analysis ===")
                
                # Analyze first demo
                first_demo = demo_keys[0]
                demo = data_group[first_demo]
                
                # Actions analysis
                if 'actions_xyz' in demo:
                    actions = demo['actions_xyz'][:]
                    print(f"\n--- Robot Actions (actions_xyz) ---")
                    print(f"Shape: {actions.shape}")
                    print(f"Value range: [{actions.min():.3f}, {actions.max():.3f}]")
                    print(f"Mean: {actions.mean():.3f}, Std: {actions.std():.3f}")
                    
                    # Plot action trajectories
                    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
                    
                    # Reshape for plotting (flatten time dimension)
                    actions_flat = actions.reshape(-1, actions.shape[-1])
                    
                    if actions.shape[-1] >= 3:
                        # Plot X, Y, Z trajectories
                        axes[0, 0].plot(actions_flat[:, 0], label='X')
                        axes[0, 0].set_title('Robot X Trajectory')
                        axes[0, 0].legend()
                        
                        axes[0, 1].plot(actions_flat[:, 1], label='Y')
                        axes[0, 1].set_title('Robot Y Trajectory')
                        axes[0, 1].legend()
                        
                        axes[1, 0].plot(actions_flat[:, 2], label='Z')
                        axes[1, 0].set_title('Robot Z Trajectory')
                        axes[1, 0].legend()
                        
                        # 3D trajectory
                        ax = axes[1, 1]
                        ax = plt.subplot(2, 2, 4, projection='3d')
                        ax.plot(actions_flat[:, 0], actions_flat[:, 1], actions_flat[:, 2])
                        ax.set_xlabel('X')
                        ax.set_ylabel('Y')
                        ax.set_zlabel('Z')
                        ax.set_title('Robot 3D Trajectory')
                    
                    plt.tight_layout()
                    plt.show()
                
                # EE poses analysis
                if 'obs/ee_pose' in demo:
                    ee_poses = demo['obs/ee_pose'][:]
                    print(f"\n--- Robot End-Effector Poses (ee_pose) ---")
                    print(f"Shape: {ee_poses.shape}")
                    print(f"Value range: [{ee_poses.min():.3f}, {ee_poses.max():.3f}]")
                    print(f"Mean: {ee_poses.mean():.3f}, Std: {ee_poses.std():.3f}")
                
                # Actions_xyz_act analysis
                if 'actions_xyz_act' in demo:
                    actions_act = demo['actions_xyz_act'][:]
                    print(f"\n--- Interpolated Robot Actions (actions_xyz_act) ---")
                    print(f"Shape: {actions_act.shape}")
                    print(f"actions_xyz_act: {actions_act[0]}")
                    print(f"Value range: [{actions_act.min():.3f}, {actions_act.max():.3f}]")
                    print(f"Mean: {actions_act.mean():.3f}, Std: {actions_act.std():.3f}")
                
                # Joint positions analysis
                if 'obs/joint_positions' in demo:
                    joint_positions = demo['obs/joint_positions'][:]
                    print(f"\n--- Robot Joint Positions ---")
                    print(f"Shape: {joint_positions.shape}")
                    print(f"Value range: [{joint_positions.min():.3f}, {joint_positions.max():.3f}]")
                    print(f"Mean: {joint_positions.mean():.3f}, Std: {joint_positions.std():.3f}")


In [6]:
if Path(hdf5_path).exists():
    with h5py.File(hdf5_path, 'r') as f:
        print("=== Robot Data Summary ===")
        
        if 'data' in f:
            data_group = f['data']
            demo_keys = list(data_group.keys())
            
            print(f"Data format: RoboMimic")
            print(f"Number of robot demos: {len(demo_keys)}")
            
            total_samples = 0
            for demo_key in demo_keys:
                demo = data_group[demo_key]
                if 'num_samples' in demo.attrs:
                    total_samples += demo.attrs['num_samples']
            
            print(f"Total robot samples: {total_samples}")
            
            # Check data consistency
            if demo_keys:
                first_demo = data_group[demo_keys[0]]
                
                print(f"\n--- Robot Data Consistency Check ---")
                
                # Check if all required datasets exist
                required_datasets = ['actions_xyz', 'obs/front_img_1', 'obs/ee_pose']
                for dataset in required_datasets:
                    if dataset in first_demo:
                        print(f"✅ {dataset}: {first_demo[dataset].shape}")
                    else:
                        print(f"❌ {dataset}: Missing")
                
                # Check optional datasets
                optional_datasets = ['actions_xyz_act', 'obs/front_img_1_masked', 'obs/front_img_1_line', 'obs/front_img_1_mask', 'obs/joint_positions']
                for dataset in optional_datasets:
                    if dataset in first_demo:
                        print(f"✅ {dataset}: {first_demo[dataset].shape}")
                    else:
                        print(f"⚠️  {dataset}: Not available")
        
        elif 'rgb' in f:
            print(f"Data format: Raw RealSense")
            print(f"RGB frames: {f['rgb'].shape[0]}")
            print(f"Depth frames: {f['depth'].shape[0]}")
            print(f"Resolution: {f['rgb'].shape[1]}x{f['rgb'].shape[2]}")
            
            if 'timestamps' in f:
                timestamps = f['timestamps'][:]
                duration = timestamps[-1] - timestamps[0]
                fps = len(timestamps) / duration
                print(f"Duration: {duration:.2f} seconds")
                print(f"FPS: {fps:.2f}")
        
        else:
            print("Unknown data format")
            print(f"Available keys: {list(f.keys())}")
        
        print("\n=== Robot Data Inspection Complete ===")
